In [1]:
import tensorflow as tf
import numpy as np
import os

context = 5
feature_dimension =39
fc_neurons = 256
tf.set_random_seed(1234)

In [2]:
x = tf.placeholder( shape = [None, feature_dimension, 2*context + 1], dtype = tf.float32)
y = tf.placeholder( shape = [None, 2], dtype = tf.float32)
keep_prob = tf.placeholder(dtype= tf.float32)

input_lstm = tf.unstack( x, 2*context +1, 2)
print(input_lstm)

[<tf.Tensor 'unstack:0' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:1' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:2' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:3' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:4' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:5' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:6' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:7' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:8' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:9' shape=(?, 39) dtype=float32>, <tf.Tensor 'unstack:10' shape=(?, 39) dtype=float32>]


In [3]:
lstm_fw = tf.contrib.rnn.LSTMCell( 10, use_peepholes = True,
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))

lstm_back = tf.contrib.rnn.LSTMCell( 5, use_peepholes = True,
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))

blstm_out, state_fw, state_bw = tf.contrib.rnn.stack_bidirectional_rnn( [lstm_fw], [lstm_back], input_lstm, dtype=tf.float32)

KeyboardInterrupt: 

In [ ]:
weights1 = tf.get_variable(name='dw1', shape = [15, fc_neurons],
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
biases1 = tf.get_variable(name='db1', shape=[fc_neurons],
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))

dense1 = tf.matmul( blstm_out[-1], weights1) + biases1
dense1_out = tf.nn.relu(dense1)
dense1_dropout = tf.nn.dropout( dense1_out, keep_prob )

In [ ]:
weights2 = tf.get_variable(name='dw2', shape = [fc_neurons, fc_neurons],
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
biases2 = tf.get_variable(name='db2', shape=[fc_neurons],
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))

dense2 = tf.matmul( dense1_dropout, weights2) + biases2
dense2_out = tf.nn.relu(dense2)
dense2_dropout = tf.nn.dropout( dense2_out, keep_prob )

In [ ]:
weights3 = tf.get_variable(name='dw3', shape = [fc_neurons, fc_neurons],
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
biases3 = tf.get_variable(name='db3', shape=[fc_neurons],
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))

dense3 = tf.matmul( dense2_dropout, weights3) + biases3
dense3_out = tf.nn.relu(dense3)
dense3_dropout = tf.nn.dropout( dense3_out, keep_prob )

In [ ]:
weights_final = tf.get_variable(name='dfw', shape = [fc_neurons, 2],
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))
biases_final = tf.get_variable(name='dfb', shape=[2],
                    initializer = tf.contrib.layers.xavier_initializer(uniform=True, seed=None, dtype=tf.float32))

output = tf.matmul( dense3_dropout, weights_final) + biases_final
output1 = tf.nn.softmax(output)

In [ ]:
cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = output))
optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(cross_entropy)

correct_predictions = tf.equal( tf.argmax(output, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean( tf.cast( correct_predictions, tf.float32 ))

In [ ]:
#initializing tensors using sessoion
session = tf.InteractiveSession()

init = tf.global_variables_initializer()
saver = tf.train.Saver(max_to_keep=None)
features_path = "K:\\MFCC\\context_5\\CNN_train_data"
development_path = "K:\\MFCC\\context_5\\CNN_test_data"
# evaluation_path = "H:\\LP_residual_phase\\CNN_input_testing_eval"
model_path = "K:\\HT-IACC_BLSTM_120\\context_5\\BLSTM_10,5_neurons_3FC_512_N\\Models\\"
    
dev_sp_path = "K:\\HT-IACC_BLSTM_120\\context_5\\BLSTM_10,5_neurons_3FC_512_N\\dev"
# eva_sp_path = "G:\\cqcc_database\\context_10\\BLSTM\\eval" 

In [ ]:
def cal_eer(scores_true, scores_false):
    
    """ Calculate EER
        Args:
            scores_true: A score list of positive samples
            scores_false: A score list of negative samples
        Return:
            (EER, threshold)
        Note:
            Here, the threshold is set, and if the score is higher than or equal to the threshold,
            the decision is passed, and the decision is rejected as below.
    """
    min1 = min(scores_true)
    min2 = min(scores_false)
    low = min(min1, min2)
    max1 = max(scores_true)
    max2 = max(scores_false)
    high = max(max1, max2)
    FAR = -55
    FRR = -55
    mid = -100
    # Bisection find threshold
    while True:
        bef_FAR = FAR 
        bef_FRR = FRR
        bef_mid = mid
        mid = (high + low) / 2
        FRR = sum([1 for s in scores_true if s < mid]) / len(scores_true)
        FAR = sum([1 for s in scores_false if s >= mid]) / len(scores_false)
        #print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)

        if abs(FRR - FAR) < 1e-7:
            print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
            break

        if FRR < FAR:
            low = mid
        else:
            high = mid
            
        if bef_FAR == FAR :
            if bef_FRR == FRR :
                if bef_mid == mid :
                    print("FRR  ",FRR," FAR   ",FAR,"   mid   ",mid)
                    break

    EER  =  FRR
    return EER, (high + low) / 2

In [ ]:
from scipy.io import savemat

def get_EER(test_path, index, sp_path ):
    scores = []
    development_labels = []
    probabilities = []

    #test_path = "C:\\Users\\project1\\Desktop\\Siva\\HT_IFCC_hemlata\\CNN_input_testing_dev"

    for folders in os.listdir(test_path):
        features = np.load(os.path.join(test_path, folders, "feature_image.npy"))
        labels = np.load(os.path.join(test_path, folders, "label_image.npy"))
        patches = labels.shape[0]
        patch_prob = session.run(output1, feed_dict = {x:features, keep_prob:0.5})
        final_prob = np.zeros([2])
        
        for i in range(patches):
            if(patch_prob[i][0] == 0):
                patch_prob[i][0] = 1e-25
            if(patch_prob[i][1] == 0):
                patch_prob[i][1] = 1e-25
                
            final_prob[0] = final_prob[0] + np.log(patch_prob[i][0])
            final_prob[1] = final_prob[1] + np.log(patch_prob[i][1])

        final_prob = final_prob / patches
        score = (final_prob[0]) - (final_prob[1])
        audio_label = labels[0]

        scores.append(score)
        probabilities.append( np.mean(patch_prob, axis=0) )
        development_labels.append( audio_label )

    scores = np.array(scores)
    development_labels = np.array(development_labels)
    probabilities = np.array(probabilities)

    scores_geniue = []
    scores_spoof = []
    prob_genuine = []
    prob_spoof = []

    for i in range(development_labels.shape[0]):
        if development_labels[i][0] == 1 :
            scores_geniue.append(scores[i])
            prob_genuine.append(probabilities[i])
        elif development_labels[i][1] == 1:
            scores_spoof.append(scores[i])
            prob_spoof.append(probabilities[i])

    scores_geniue = np.array(scores_geniue)
    scores_spoof = np.array(scores_spoof)
    prob_genuine = np.array(prob_genuine)
    prob_spoof = np.array(prob_spoof)
    
    a,b = cal_eer(scores_geniue, scores_spoof)
    
    score_path = os.path.join(sp_path, "scores")
    prob_path = os.path.join(sp_path, "probabilities")
    
    if not os.path.exists(score_path):
        os.makedirs(score_path)
    if not os.path.exists(prob_path):
        os.makedirs(prob_path)
        
    prob_file = "probabilities_"+str(index)
    sc_file = "scores_"+str(index)
    savemat(os.path.join(score_path, sc_file), { 'genuine':scores_geniue, 'spoof':scores_spoof, 'eer' : a*100 })
    savemat(os.path.join(prob_path, prob_file), { 'genuine':prob_genuine, 'spoof':prob_spoof })
    
    return a

In [ ]:
#Training and testing accuracy
session.run(init)

if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists(dev_sp_path):
    os.makedirs(dev_sp_path)
    
batches = (len(list(os.listdir(features_path))))//2
    
for i in range(0,100):
    for j in range(0,batches):  #number of batches
        filename = "batch_"+str(j)+"_features.npy"
        labelsname= "batch_"+str(j)+"_labels.npy"
        
        features = np.load(os.path.join(features_path, filename))
        labels = np.load(os.path.join(features_path, labelsname))

        train_step.run(session = session,feed_dict={ x:features, y:labels, keep_prob:0.5})
        
   
    print("iteration   ",(i+1),"\n")
#     test_accuracy = accuracy.eval(session = session, feed_dict = { x:test_features, y:test_labels })
#     print("iteration   ",(i+1),"   dev_accuracy    ",test_accuracy)    
#     print()
    save_path = saver.save(session, model_path +str(i))
    

In [ ]:
saver = tf.train.Saver(max_to_keep=None)

with tf.Session() as session:
    for i in range(0,100):
        saver.restore(session, model_path +str(i))
        print("iteration  ",(i+1))
        eer_dev = get_EER(development_path, (i+1), dev_sp_path)
        print(" develop eer  ",eer_dev)
        print()